In [1]:
# Install required packages for constellation recognition UI

import ipywidgets
from IPython.display import display, HTML, clear_output
from ultralytics import YOLO
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
from PIL import Image, ExifTags
import io
import base64

# Define the dataset path
DATASET_PATH = "constellation_dataset"

In [2]:
# Load the YOLOv8 model
def load_model(model_path="best_constellation_model.pt"):
    """Load the YOLOv8 constellation recognition model"""
    try:
        model = YOLO(model_path)
        print(f"Successfully loaded model from {model_path}")
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

# Process image for constellation recognition
def recognize_constellation(img, model):
    """Recognize constellation in an image using YOLOv8 model"""
    if model is None:
        return img, "Error: Model not loaded", None
    
    # Make a copy for annotation
    display_img = img.copy()
    
    # Run inference
    results = model(img)
    
    # Initialize variables for storing results
    recognized_constellation = "No constellation detected"
    confidence = 0
    
    # Process results
    if len(results) > 0:
        # Get the first result (assuming single image)
        result = results[0]
        
        # Check if any detections
        if len(result.boxes) > 0:
            # Get the detection with highest confidence
            confidences = result.boxes.conf.cpu().numpy()
            class_ids = result.boxes.cls.int().cpu().numpy()
            boxes = result.boxes.xyxy.cpu().numpy()
            
            # Get the index of highest confidence detection
            best_idx = np.argmax(confidences)
            
            # Get the class name and confidence
            class_id = class_ids[best_idx]
            class_name = result.names[class_id]
            confidence = confidences[best_idx]
            
            # Set recognized constellation
            recognized_constellation = class_name
            
            # Draw bounding box on the image
            box = boxes[best_idx].astype(int)
            cv2.rectangle(display_img, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
            cv2.putText(display_img, f"{class_name} {confidence:.2f}", 
                       (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    # Convert the annotated image from BGR to RGB for display
    display_img_rgb = cv2.cvtColor(display_img, cv2.COLOR_BGR2RGB)
    
    return display_img_rgb, recognized_constellation, confidence

In [3]:
from PIL import Image, ExifTags
import cv2
import numpy as np

def preprocess_image(image_input, output_size=(640, 640)):
    """
    Preprocess an image by auto-orienting, resizing it to 640x640 (stretch), and converting it to grayscale.

    Args:
        image_input: Either a path to an image file (str) or an image array (numpy array)
        output_size (tuple): Desired output size (width, height).

    Returns:
        processed_image: The preprocessed grayscale image.
    """
    # Check if input is a path or an array
    if isinstance(image_input, str):
        # Load the image using PIL to handle EXIF orientation
        pil_image = Image.open(image_input)
        
        # Auto-orient the image based on EXIF metadata
        try:
            for orientation in ExifTags.TAGS.keys():
                if ExifTags.TAGS[orientation] == 'Orientation':
                    break
            exif = pil_image._getexif()
            if exif is not None:
                orientation_value = exif.get(orientation, None)
                if orientation_value == 3:
                    pil_image = pil_image.rotate(180, expand=True)
                elif orientation_value == 6:
                    pil_image = pil_image.rotate(270, expand=True)
                elif orientation_value == 8:
                    pil_image = pil_image.rotate(90, expand=True)
        except Exception as e:
            print(f"Warning: Could not auto-orient image due to: {e}")
        
        # Convert PIL image to OpenCV format (numpy array)
        image = np.array(pil_image)
    else:
        # Input is already an image array
        image = image_input.copy()

    # If the image has an alpha channel, remove it
    if len(image.shape) == 3 and image.shape[2] == 4:
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)

    # Ensure image is in RGB format
    if len(image.shape) == 3 and image.shape[2] == 3:
        # Resize the image to 640x640 (stretch)
        resized_image = cv2.resize(image, output_size, interpolation=cv2.INTER_LINEAR)
        
        # Convert the resized image to grayscale
        grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    else:
        # Image is already grayscale, just resize
        resized_image = cv2.resize(image, output_size, interpolation=cv2.INTER_LINEAR)
        grayscale_image = resized_image

    return grayscale_image

In [4]:
def create_constellation_recognition_ui():
    """Create an interactive UI for constellation recognition"""
    from ipywidgets import widgets, HBox, VBox, Layout, Output
    from IPython.display import display, HTML, clear_output
    from io import BytesIO
    
    # Define the layout for full-width widgets
    full_width = Layout(width='100%')
    
    # Create output widget for displaying results
    output_widget = widgets.Output()
    
    # Create file upload widget
    file_upload = widgets.FileUpload(
        accept='.jpg, .jpeg, .png',
        multiple=False,
        description='Upload Image:',
        layout=Layout(width='auto')
    )
    
    # Enhancement options
    enhance_image = widgets.Checkbox(
        value=True,
        description='Enhance stars before recognition',
        disabled=False
    )
    
    # Preprocessing options
    preprocess_image_checkbox = widgets.Checkbox(
        value=False,
        description='Apply preprocessing (auto-orient, resize to 640x640)',
        disabled=False
    )
    
    # Create process button
    process_button = widgets.Button(
        description='Recognize Constellation',
        button_style='primary',
        tooltip='Process the image with the constellation recognition model',
        layout=Layout(width='auto')
    )

    # Sample images dropdown
    sample_images_dir = os.path.join(DATASET_PATH, "test", "images")
    sample_image_options = ["Select a sample image..."]
    
    if os.path.exists(sample_images_dir):
        sample_files = [f for f in os.listdir(sample_images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        # Include only first 20 samples to avoid huge dropdown
        sample_image_options.extend(sample_files[:20])
    
    sample_dropdown = widgets.Dropdown(
        options=sample_image_options,
        value=sample_image_options[0],
        description='Or try a sample:',
        disabled=False,
        layout=full_width
    )
    
    # Load YOLOv8 model
    model = load_model()
    
    # Function to handle image processing
    def process_image(b):
        with output_widget:
            clear_output()
            
            # Show loading message
            print("Processing image...")
            
            img = None
            source_info = ""
            original_img = None # Store the original image for comparison
            
            # Check if file was uploaded
            if len(file_upload.value) > 0:
                # Get the uploaded file
                uploaded_file = file_upload.value[0]
                content = uploaded_file.content
                filename = uploaded_file.name
                
                if preprocess_image_checkbox.value:
                    # Save uploaded image to a temporary file to use with preprocess_image
                    temp_file = f"temp_{filename}"
                    with open(temp_file, 'wb') as f:
                        f.write(content)
                    
                    # Apply preprocessing
                    processed = preprocess_image(temp_file)
                    
                    # Convert grayscale back to BGR for further processing
                    img = cv2.cvtColor(processed, cv2.COLOR_GRAY2BGR)
                    
                    # Save original for comparison
                    img_array = np.frombuffer(content, dtype=np.uint8)
                    original_img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
                    
                    # Clean up temp file
                    try:
                        os.remove(temp_file)
                    except:
                        pass
                    
                    source_info = f"Uploaded image: {filename} (preprocessed)"
                else:
                    # Standard loading without preprocessing
                    img_array = np.frombuffer(content, dtype=np.uint8)
                    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
                    source_info = f"Uploaded image: {filename}"
                    # Save the original image for later use if enhancement is requested
                    original_img = img.copy()
                
            # Alternatively, use the selected sample image
            elif sample_dropdown.value != sample_image_options[0]:
                img_path = os.path.join(sample_images_dir, sample_dropdown.value)
                
                if preprocess_image_checkbox.value:
                    # Apply preprocessing
                    processed = preprocess_image(img_path)
                    
                    # Convert grayscale back to BGR for further processing
                    img = cv2.cvtColor(processed, cv2.COLOR_GRAY2BGR)
                    
                    # Save original for comparison
                    original_img = cv2.imread(img_path)
                    
                    source_info = f"Sample image: {sample_dropdown.value} (preprocessed)"
                else:
                    # Standard loading without preprocessing
                    img = cv2.imread(img_path)
                    source_info = f"Sample image: {sample_dropdown.value}"
            
            if img is not None:
                # If we preprocessed, show both original and preprocessed images
                if preprocess_image_checkbox.value and original_img is not None:
                    plt.figure(figsize=(12, 5))
                    plt.subplot(1, 2, 1)
                    plt.imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
                    plt.title("Original Image")
                    plt.axis('off')
                    
                    plt.subplot(1, 2, 2)
                    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                    plt.title("Preprocessed Image")
                    plt.axis('off')
                    plt.tight_layout()
                    plt.show()
                
                # Enhance image if requested
                if enhance_image.value:
                    # Create directories if they don't exist
                    os.makedirs("raw_images", exist_ok=True)
                    os.makedirs("processed_images", exist_ok=True)
                    
                    # Generate a unique filename with timestamp
                    timestamp = int(time.time())
                    filename = f"image_{timestamp}.jpg"
                    raw_image_path = os.path.join("raw_images", filename)
                    processed_image_path = os.path.join("processed_images", filename)
                    
                    # Save the original image to raw_images folder
                    cv2.imwrite(raw_image_path, img)
                    
                    # Process the image with preprocess_image function
                    processed = preprocess_image(raw_image_path)
                    
                    # Convert grayscale back to BGR for further processing
                    processed_bgr = cv2.cvtColor(processed, cv2.COLOR_GRAY2BGR)
                    
                    # Save the processed image to processed_images folder
                    cv2.imwrite(processed_image_path, processed_bgr)
                    
                    # Use the processed image for recognition
                    img = processed_bgr
                
                # Process the image with the model
                start_time = time.time()
                result_img, constellation, confidence = recognize_constellation(img, model)
                end_time = time.time()
                
                # Display results
                plt.figure(figsize=(12, 8))
                plt.imshow(result_img)
                plt.axis('off')
                plt.title(f"Detected: {constellation}" + (f" (Confidence: {confidence:.2f})" if confidence else ""))
                plt.tight_layout()
                plt.show()
                
                # Display additional information
                # Fix string formatting issue by handling the confidence formatting separately
                confidence_str = f"{confidence:.2f}" if confidence else "N/A"
                
                # Include image paths in the result information if enhancement was applied
                image_paths_info = ""
                if enhance_image.value:
                    image_paths_info = f"""
                    <p><strong>Raw Image Path:</strong> {raw_image_path}</p>
                    <p><strong>Processed Image Path:</strong> {processed_image_path}</p>
                    """
                
                result_info = f"""
                <div style="background-color: #f5f5f5; padding: 15px; border-radius: 5px; margin-top: 20px;">
                    <h3 style="margin-top: 0;">Recognition Results</h3>
                    <p><strong>Source:</strong> {source_info}</p>
                    <p><strong>Detected Constellation:</strong> {constellation}</p>
                    <p><strong>Confidence:</strong> {confidence_str}</p>
                    <p><strong>Processing Time:</strong> {end_time - start_time:.2f} seconds</p>
                    <p><strong>Enhancement Applied:</strong> {'Yes' if enhance_image.value else 'No'}</p>
                    <p><strong>Preprocessing Applied:</strong> {'Yes' if preprocess_image_checkbox.value else 'No'}</p>
                    {image_paths_info}
                </div>
                """
                display(HTML(result_info))
                
                # Print a brief explanation of the constellation
                constellations_info = {
                    'Aquarius': "The Water Bearer constellation, representing Ganymede, a handsome young man who was carried to Olympus by Zeus disguised as an eagle.",
                    'Aries': "The Ram constellation, representing the ram with the Golden Fleece from Greek mythology.",
                    'Cancer': "The Crab constellation, representing the crab that Hera sent to distract Heracles during his fight with the Hydra.",
                    'Capricornus': "The Sea Goat constellation, representing Pan who transformed into a half-goat, half-fish when escaping the monster Typhon.",
                    'Gemini': "The Twins constellation, representing Castor and Pollux, the twin sons of Leda and Zeus.",
                    'Leo': "The Lion constellation, representing the Nemean Lion slain by Heracles as one of his twelve labors.",
                    'Libra': "The Scales constellation, representing the scales of justice held by the goddess Astraea (Virgo).",
                    'Pisces': "The Fish constellation, representing Aphrodite and Eros who transformed into fish to escape Typhon.",
                    'Sagittarius': "The Archer constellation, representing a centaur, usually identified as Chiron, who was accidentally wounded by Heracles.",
                    'Scorpius': "The Scorpion constellation, representing the scorpion that killed Orion the Hunter.",
                    'Taurus': "The Bull constellation, representing Zeus when he took the form of a bull to seduce Europa.",
                    'Virgo': "The Maiden constellation, representing several goddesses including Demeter, Persephone, and Astraea."
                }
                
                if constellation in constellations_info:
                    print(f"\n📚 About {constellation}:")
                    print(constellations_info[constellation])
                
            else:
                print("Please upload an image or select a sample image")
    
    # Function to handle sample selection
    def on_sample_change(change):
        if change['new'] != sample_image_options[0]:
            # Clear any uploaded files
            file_upload.value = ()
    
    # Function to handle file upload
    def on_file_upload(change):
        if len(file_upload.value) > 0:
            # Reset sample dropdown
            sample_dropdown.value = sample_image_options[0]
    
    # Connect event handlers
    process_button.on_click(process_image)
    sample_dropdown.observe(on_sample_change, names='value')
    file_upload.observe(on_file_upload, names='value')
    
    # Create layout
    upload_section = VBox([
        widgets.HTML("<h3>Step 1: Select an Image</h3>"),
        HBox([file_upload]),
        widgets.HTML("<p>OR</p>"),
        sample_dropdown
    ])
    
    options_section = VBox([
        widgets.HTML("<h3>Step 2: Options</h3>"),
        preprocess_image_checkbox,
        enhance_image
    ])
    
    process_section = VBox([
        widgets.HTML("<h3>Step 3: Process</h3>"),
        process_button
    ])
    
    # Main layout
    main_layout = VBox([
        widgets.HTML("<h2>Constellation Recognition</h2>"),
        HBox([upload_section, options_section, process_section]),
        widgets.HTML("<hr>"),
        output_widget
    ])
    
    # Display the UI
    display(main_layout)

# Initialize the UI
print("Initializing Constellation Recognition UI...")
create_constellation_recognition_ui()

Initializing Constellation Recognition UI...
Successfully loaded model from best_constellation_model.pt
